In [1]:
import boto3
import re
import datetime
from io import BytesIO
import os
from PIL import Image

# Clase OCR

In [2]:
class TextractOCR:
    def __init__(self, aws_region="us-east-1"):
        self.textract = boto3.client("textract", region_name=aws_region)

    def extract_text_from_file(self, file_path):
        """Extrae texto plano línea por línea desde una imagen"""
        with open(file_path, "rb") as image_file:
            image_bytes = image_file.read()

        response = self.textract.detect_document_text(Document={'Bytes': image_bytes})
        
        raw_text = ""
        for item in response["Blocks"]:
            if item["BlockType"] == "LINE":
                raw_text += item["Text"] + "\n"

        return raw_text
    

    def extract_text_from_image(self, i):
        image = Image.open(BytesIO(i))
        buffer = BytesIO()
        image.save(buffer, format="PNG")  # Asegúrate que sea PNG si ese es el formato real
        image_bytes = buffer.getvalue()

        response = self.textract.detect_document_text(Document={'Bytes': image_bytes})

        raw_text = ""
        for item in response["Blocks"]:
            if item["BlockType"] == "LINE":
                raw_text += item["Text"] + "\n"

        return raw_text


    def extract_and_save(self, file_path, output_txt_path):
        """Extrae texto y lo guarda en un archivo .txt"""
        text = self.extract_text_from_file(file_path)
        with open(output_txt_path, "w", encoding="utf-8") as f:
            f.write(text)
        return text


# Implementación OCR

In [3]:
ocr = TextractOCR()

In [4]:
raw_text = ocr.extract_text_from_file("../Cadena/FT Prueba Concepto/290601-R.jpg")

In [5]:
print(raw_text)

1
Nombre:
04-07-2025
Cédula:
0000004
2
HpqwOD+
10
Teléfono:
Firma:
5-2000
Lotería del
Risaralda
FRACCIÓN $4.000 - BILLETE $12.000
00000000
5
PLAN DE PREMIOS DE LA LOTERÍA DEL RISARALDA
PREMIOESPECIAL
CONCEPTO
LA ESCALERA MILLONARIA
3
(Terminos condiciones www loteriadelrisaralda.com)
JUEGA EL DE
PREMIO MAYOR CON SERIE
1.549.333.333
8
Acertando los cuatro números del sorteo en orden mas la
JULIO DE 2025
PREMIOS SECOS CON SERIE
serie en orden del (NUMERO PRINCIPAL)
39,840,000
Risaralda
EL GORDO DE LA RISARALDA
199.200.000
ANGEL DE LA SUERTE
132.800.000
Aproximacion Escalera Millonaria
MULA MIL ONARIA
99,600.000
Acertando cifras en cualquier order mas serie
5
MILAGRO MILLONARIO
53,120,000
del NUMERO PRINCIP del billete resultados
2,294,971
GUACA DE ORO
39.840.000
del sorteo (Escalera Millonaria).
COFRE DE DIAMANTES
33.200.000
TREBOL DE LA FORTUNA
6
26.560.000
Aproximacion Premio Escalera Vilonaria Acertando cifras
SECOS
DE
23.240.000
del NÚMERO en cadabillete e su orden mas
2,523,200
BILL

# Clase Validaciones

In [36]:

class Validaciones:
    def __init__(self, raw_text: str, sorteo: str, fecha_sorteo: str, dia_de_juego: str, hora_de_juego: str, premio_mayor: str, valor_billete: str, valor_fraccion: str, serie: str, numero: str, numero_escrito: list):
        self.text = self._preprocesar(raw_text)
        self.sorteo = sorteo.lower()
        self.fecha_sorteo = fecha_sorteo.lower()
        self.dia_de_juego = dia_de_juego.lower()
        self.hora_de_juego = hora_de_juego.lower()
        self.premio_mayor = premio_mayor.lower()
        self.valor_billete = valor_billete.lower()
        self.valor_fraccion = valor_fraccion.lower()
        self.serie = serie.lower()
        self.numero = numero.lower()
        self.numero_escrito = [p.lower().strip() for p in numero_escrito if p.strip()]
    
    def _preprocesar(self, texto: str) -> str:
        # Convierte a minúsculas, elimina saltos de línea y múltiples espacios
        texto = texto.lower()
        texto = texto.replace("\n", " ")
        texto = re.sub(r"\s+", " ", texto)  # Remueve espacios duplicados
        return texto    

    def check_sorteo(self):
        return self.sorteo in self.text
    
    def check_fecha(self):
        # Acepta variantes como "15 de abril", "abril 15", etc.
        fecha_variantes = [
            self.fecha_sorteo,
            re.sub(r"(\d{1,2}) de (\w+)", r"\2 \1", self.fecha_sorteo),  # "15 de abril" -> "abril 15"
        ]
        return any(var in self.text for var in fecha_variantes)

    def check_dia(self):
        return self.dia_de_juego in self.text

    def check_hora(self):
        return self.hora_de_juego in self.text

    def check_premio_mayor(self):
        return self.premio_mayor in self.text

    def check_valor_billete(self):
        return self.valor_billete in self.text

    def check_valor_fraccion(self):
        return self.valor_fraccion in self.text

    def check_serie(self):
        return self.serie in self.text
    
    def check_numero(self):
        return self.numero in self.text
    
    def check_numero_escrito(self):
        return all(palabra in self.text for palabra in self.numero_escrito)

    def run_all_checks(self):
        return {
            "sorteo": self.check_sorteo(),
            "fecha": self.check_fecha(),
            "dia": self.check_dia(),
            "hora": self.check_hora(),
            "premio_mayor": self.check_premio_mayor(),
            "valor_billete": self.check_valor_billete(),
            "valor_fraccion": self.check_valor_fraccion(),
            "serie": self.check_serie(),
            "numero": self.check_numero(),
            "numero_escrito": self.check_numero_escrito()
        }
    
    def count_sorteo(self):
        return self.text.count(self.sorteo)

    def count_fecha(self):
        variantes = set([
            self.fecha_sorteo,
            re.sub(r"(\d{1,2}) de (\w+)", r"\2 \1", self.fecha_sorteo),  # "15 de abril" → "abril 15"
            re.sub(r"(\d{1,2})/(\d{1,2})/(\d{4})", r"\1/\2", self.fecha_sorteo),  # "15/04/2025" → "15/04"
        ])
        return sum(self.text.count(var) for var in variantes if var)

    def count_dia(self):
        return self.text.count(self.dia_de_juego)

    def count_hora(self):
        return self.text.count(self.hora_de_juego)

    def count_premio_mayor(self):
        return self.text.count(self.premio_mayor)

    def count_valor_billete(self):
        return self.text.count(self.valor_billete)

    def count_valor_fraccion(self):
        return self.text.count(self.valor_fraccion)

    def count_serie(self):
        return self.text.count(self.serie)
    
    def count_numero(self):
        return self.text.count(self.numero)

    def count_numero_escrito(self):
        if len(self.numero_escrito) == 1:
            return self.text.count(self.numero_escrito[0])
        
        palabras_texto = self.text.split()
        count = 0
        window_size = len(self.numero_escrito) + 3  # Puedes ajustar la ventana según precisión/difusión

        for i in range(len(palabras_texto) - window_size + 1):
            ventana = palabras_texto[i:i + window_size]
            if all(palabra in ventana for palabra in self.numero_escrito):
                count += 1

        return count

    def run_all_counts(self):
        return {
            "sorteo": self.count_sorteo(),
            "fecha": self.count_fecha(),
            "dia": self.count_dia(),
            "hora": self.count_hora(),
            "premio_mayor": self.count_premio_mayor(),
            "valor_billete": self.count_valor_billete(),
            "valor_fraccion": self.count_valor_fraccion(),
            "serie": self.count_serie(),
            "numero": self.count_numero(),
            "numero_escrito": self.count_numero_escrito()
        }


# Implementación Validaciones

In [56]:
validador = Validaciones(
    raw_text=raw_text,
    sorteo="3097",
    fecha_sorteo="15 de Abril",
    dia_de_juego="martes",
    hora_de_juego="10:55 p.m.",
    premio_mayor="7.000",
    valor_billete="$15.000",
    valor_fraccion="$7.500",
    serie="244",
    numero="2023",
    numero_escrito=["veinte", "veintitres"]
)

resultados = validador.run_all_checks()
counts = validador.run_all_counts()

In [57]:
print("Resultados de las validaciones:")
for key, value in resultados.items():
    print(f"{key}: {value}")


print("\nCounts:")
for key, value in counts.items():
    print(f"{key} count: {value}")

Resultados de las validaciones:
sorteo: True
fecha: True
dia: True
hora: True
premio_mayor: True
valor_billete: True
valor_fraccion: True
serie: True
numero: True
numero_escrito: True

Counts:
sorteo count: 4
fecha count: 2
dia count: 3
hora count: 3
premio_mayor count: 1
valor_billete count: 3
valor_fraccion count: 3
serie count: 3
numero count: 3
numero_escrito count: 4


# AWS Rekognition

In [1]:
import boto3
from io import BytesIO
from PIL import Image

class RekognitionVisualValidator:
    def __init__(self, aws_region="us-east-1"):
        self.rekognition = boto3.client("rekognition", region_name=aws_region)

    def detect_labels_from_file(self, file_path, max_labels=10, min_confidence=80):
        """
        Detecta objetos/elementos visuales (labels) en la imagen.
        Devuelve una lista de diccionarios con nombre, confianza y coordenadas (si aplica).
        """
        with open(file_path, "rb") as image_file:
            image_bytes = image_file.read()
        
        response = self.rekognition.detect_labels(
            Image={'Bytes': image_bytes},
            MaxLabels=max_labels,
            MinConfidence=min_confidence
        )
        
        results = []
        for label in response['Labels']:
            entry = {
                "Name": label["Name"],
                "Confidence": label["Confidence"],
                "Instances": []
            }
            # Agrega las coordenadas (bounding box) si están disponibles
            for instance in label.get("Instances", []):
                if "BoundingBox" in instance:
                    entry["Instances"].append(instance["BoundingBox"])
            results.append(entry)
        return results

    def detect_text_from_file(self, file_path, min_confidence=70):
        """
        Detecta texto en la imagen y devuelve una lista con el texto detectado,
        el tipo de texto (línea, palabra) y la caja delimitadora.
        """
        with open(file_path, "rb") as image_file:
            image_bytes = image_file.read()
        
        response = self.rekognition.detect_text(
            Image={'Bytes': image_bytes}
        )
        
        results = []
        for item in response["TextDetections"]:
            if item["Confidence"] >= min_confidence:
                results.append({
                    "DetectedText": item["DetectedText"],
                    "Type": item["Type"],   # LINE o WORD
                    "BoundingBox": item["Geometry"]["BoundingBox"],
                    "Confidence": item["Confidence"]
                })
        return results

    def save_detected_labels(self, file_path, output_txt_path):
        """
        Extrae y guarda los labels/objetos detectados en un archivo de texto.
        """
        labels = self.detect_labels_from_file(file_path)
        with open(output_txt_path, "w", encoding="utf-8") as f:
            for label in labels:
                f.write(f"Label: {label['Name']}, Confianza: {label['Confidence']}\n")
                for bbox in label["Instances"]:
                    f.write(f"\tBoundingBox: {bbox}\n")
        return labels

    def save_detected_text(self, file_path, output_txt_path):
        """
        Extrae y guarda el texto detectado (líneas y palabras) en un archivo de texto.
        """
        texts = self.detect_text_from_file(file_path)
        with open(output_txt_path, "w", encoding="utf-8") as f:
            for item in texts:
                f.write(f"{item['Type']}: {item['DetectedText']} (Confianza: {item['Confidence']})\n")
                f.write(f"\tBoundingBox: {item['BoundingBox']}\n")
        return texts


In [2]:
# USO BÁSICO:
validator = RekognitionVisualValidator()
labels = validator.detect_labels_from_file("../Cadena/FT Formato PNG/FT1.png")
texts = validator.detect_text_from_file("../Cadena/FT Formato PNG/FT1.png")

In [ ]:
for label in labels:
    print(label)

{'Name': 'Advertisement', 'Confidence': 99.99813842773438, 'Instances': []}
{'Name': 'Poster', 'Confidence': 99.99620819091797, 'Instances': []}
{'Name': 'Adult', 'Confidence': 96.66357421875, 'Instances': [{'Width': 0.11092202365398407, 'Height': 0.19827638566493988, 'Left': 0.36942070722579956, 'Top': 0.4759446084499359}]}
{'Name': 'Female', 'Confidence': 96.66357421875, 'Instances': [{'Width': 0.11092202365398407, 'Height': 0.19827638566493988, 'Left': 0.36942070722579956, 'Top': 0.4759446084499359}]}
{'Name': 'Person', 'Confidence': 96.66357421875, 'Instances': [{'Width': 0.11092202365398407, 'Height': 0.19827638566493988, 'Left': 0.36942070722579956, 'Top': 0.4759446084499359}, {'Width': 0.2046440690755844, 'Height': 0.27117565274238586, 'Left': 0.4683397114276886, 'Top': 0.3662528395652771}]}
{'Name': 'Woman', 'Confidence': 96.66357421875, 'Instances': [{'Width': 0.11092202365398407, 'Height': 0.19827638566493988, 'Left': 0.36942070722579956, 'Top': 0.4759446084499359}]}
{'Name':

In [6]:
for texts in texts:
    print(texts)

{'DetectedText': '7', 'Type': 'LINE', 'BoundingBox': {'Width': 0.0087890625, 'Height': 0.010447485372424126, 'Left': 0.8525390625, 'Top': 0.010447485372424126}, 'Confidence': 95.00759887695312}
{'DetectedText': '0000000', 'Type': 'LINE', 'BoundingBox': {'Width': 0.0693359375, 'Height': 0.009794517420232296, 'Left': 0.1162109375, 'Top': 0.018283098936080933}, 'Confidence': 98.66030883789062}
{'DetectedText': '000', 'Type': 'LINE', 'BoundingBox': {'Width': 0.0224609375, 'Height': 0.007835613563656807, 'Left': 0.2041015625, 'Top': 0.020242001861333847}, 'Confidence': 98.81232452392578}
{'DetectedText': '0000', 'Type': 'LINE', 'BoundingBox': {'Width': 0.0302734375, 'Height': 0.007182646077126265, 'Left': 0.2421875, 'Top': 0.020242001861333847}, 'Confidence': 98.37816619873047}
{'DetectedText': '*0000000*', 'Type': 'LINE', 'BoundingBox': {'Width': 0.08203125, 'Height': 0.009794517420232296, 'Left': 0.31640625, 'Top': 0.018936067819595337}, 'Confidence': 99.53070831298828}
{'DetectedText': '